# 生成用于训练TimeGAN的数据

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

forex_file = 'forex.csv'
data = pd.read_csv(forex_file)  # slug, data, open, high, low, close, currency

# （相对日期，open1, high1, low1, close1, open2, high2, low2, close2, open3, high3, low3, close3)
# target: (open4, high4, low4, close4, open5, high5, low5, close5)
currency_convert_types = list(set(data['slug']))
currency_types = set()
for line in data['slug']:
    currency_types.add(line.split('/')[0])
    currency_types.add(line.split('/')[1])
currency_types = list(currency_types)

In [2]:
import torch
cur_slug = 'USD/CNY'  # 选择美元兑换为人民币
# 首先统计有多少个月份：
year_months = list(set([date[:7] for date in data['date']]))
n_month = len(year_months)
print(f'共有{n_month}个月份')
year_months.sort()
month2idx = {year_months[i]: i for i in range(n_month)}

month_record_cnt = [0] * n_month
for slug, date in zip(data['slug'], data['date']):
    if slug == cur_slug:
        month = date[:7]
        month_idx = month2idx[month]
        month_record_cnt[month_idx] += 1
print(f'去除前有{n_month}个月')
thead = 20  # 大于20天以上的才进行选择
invalid_month = []
for month, idx in month2idx.items():
    if month_record_cnt[idx] < thead:
        invalid_month.append(month)
for month in invalid_month:
    month2idx.pop(month)
year_months = list(month2idx.keys())
year_months.sort()
n_month = len(year_months)
month2idx = {year_months[i]: i for i in range(n_month)}
print(f'去除后有{n_month}个月')
output_f = './timegan-pytorch/data/USD2CNY.csv'
f = open(output_f, 'w')

共有299个月份
去除前有299个月
去除后有219个月


In [3]:
data_ori = [[[0] * 5 for day in range(31)] for month in range(n_month)]  # [n_month, 31, 4]  219, 31, 5 -> [open,high,low,close,date]

def to_str(array):
    ans = []
    for elem in array:
        ans.append(str(elem))
    return ans

for slug, date, open, high, low, close in zip(data['slug'], data['date'], data['open'], data['high'], data['low'], data['close']):
    if slug == cur_slug:
        month = date[:7]
        if month in month2idx.keys():
            month_idx = month2idx[month]
            day = int(date[-2:]) - 1
            data_ori[month_idx][day] = [open,high,low,close,date]

# 保存数据
# 第一行序号 第二行月份的编号 第三行 open 第四行 high 第五行 low 第六行 close


header = ',Idx,Open,High,Low,Close,Date'
f.write(header + '\n')

i = 0
for month_idx, month_data in enumerate(data_ori):
    for day_data in month_data:
        if day_data[:4] != [0,0,0,0]:
            line = str(i) + ',' + str(month_idx) + ',' + ','.join(to_str(day_data)) + '\n'
            f.write(line)
            i += 1